In [1]:
import requests

KBQA_URL = 'http://kbqa-service/HEALS/api/v1.0'
ENTITY_RESOLVER_URL = "http://fuzzy-entity-resolver/resolve"

In [2]:
import json
#
# Load user data from a json file
#
with open('1.json') as persona_file:
    persona_json = persona_file.read()
    persona = json.loads(persona_json)

In [3]:

# Set the base query
#
#query = "what is a breakfast without eggs"
#query = "What are soul recipes that do not contain egg?"
# query = "What are swedish recipes which do not have ingredient bread flour?"
query = "What beef ribs dishes can I make that do not contain barbecue sauce?"

In [4]:
#
# Expand the base query using user data
#
expanded_query_res = requests.post(f'{KBQA_URL}/expand_query', json={'question': query, 'persona_num': 1, 'persona': persona_json})
expanded_query = expanded_query_res.text
expanded_query

'What beef ribs dishes can I make that do not contain barbecue sauce, and have milk,  apple,  pasta or  pizza, and contain calories from carbohydrates with desired range 0.0% to 45.0%?'

In [5]:
#
# Call the entity resolver to identify keywords in the query
#
entity_resp = requests.get(ENTITY_RESOLVER_URL, params={'query': query}).json()
tags = entity_resp['tags']

print(tags)

# Filter out tags with low confidence
tags = [t for t in tags if t['confidence'] >= 0.9]
for t in tags:
    t['confidence'] = 1
#tags = [{'confidence': 1, 'entity': 'tag', 'label': 'swedish', 'value': 'tag2178'}]
tags 

[{'confidence': 0.9, 'entity': 'tag', 'label': 'beef-ribs', 'value': 'tag2239'}, {'confidence': 0.86, 'entity': 'tag', 'label': 'beef-sausage', 'value': 'tag2128'}, {'confidence': 0.86, 'entity': 'tag', 'label': 'pork-ribs', 'value': 'tag2146'}, {'confidence': 0.86, 'entity': 'tag', 'label': 'beef-organ-meats', 'value': 'tag2233'}, {'confidence': 0.86, 'entity': 'tag', 'label': 'beef-liver', 'value': 'tag2249'}]


[{'confidence': 1, 'entity': 'tag', 'label': 'beef-ribs', 'value': 'tag2239'}]

In [6]:
constrained_entities = {
      "1": [
        "thai sweet chili sauce",
        "calories from carbohydrates with desired range 35.0 % to 65.0 %"
      ],
      "2": [
        "catsup",
        "beef broth",
        "barbecue sauce"
      ]
    }


json.dumps(constrained_entities)

'{"1": ["thai sweet chili sauce", "calories from carbohydrates with desired range 35.0 % to 65.0 %"], "2": ["catsup", "beef broth", "barbecue sauce"]}'

In [7]:
persona =   {
    "ingredient_likes": [
      "thai sweet chili sauce"
    ],
    "ingredient_dislikes": [
      "catsup",
      "beef broth"
    ],
    "constrained_entities": {
      "1": [
        "thai sweet chili sauce",
        "calories from carbohydrates with desired range 35.0 % to 65.0 %"
      ],
      "2": [
        "catsup",
        "beef broth",
        "barbecue sauce"
      ]
    }
  }
#expanded_query = "What are swedish recipes which do not have ingredient bread flour, and have caraway seeds, and do not have bacon, and contain calories from saturated fat with desired range 10.0 % to 35.0 %?"
expanded_query = "What beef ribs dishes can I make that do not contain barbecue sauce, and have thai sweet chili sauce, and do not have catsup, beef broth, and contain calories from carbohydrates with desired range 35.0 % to 65.0 %?"
f'{expanded_query=}'

"expanded_query='What beef ribs dishes can I make that do not contain barbecue sauce, and have thai sweet chili sauce, and do not have catsup, beef broth, and contain calories from carbohydrates with desired range 35.0 % to 65.0 %?'"

In [17]:
#
# Query KBQA using the entities and expanded query
#
kbqa_query_data = {
    "intent": "constraint",
    "output": "N/A",
    "question": expanded_query,
    # we don't need to expand the query again
    "skip_expansion": True,
    "lookup_recipes": True,
    "persona": persona,
    "entities": tags,
    "constrained_entities": constrained_entities,
    #"similar_recipes": similar_recipes
}
kbqa_resp = requests.post(f"{KBQA_URL}/kbqa", json=kbqa_query_data)
kbqa_answer = kbqa_resp.json()
kbqa_answer

{'answer_list': ['Oven Baked Ribs'],
 'rel_path_list': [['tagged_dishes']],
 'answer_id_list': ['http://idea.rpi.edu/heals/kb/recipe/bd4767eb-Oven%20Baked%20Ribs'],
 'answer_recipe_list': [{'name': 'Oven Baked Ribs',
   'uri': 'http://idea.rpi.edu/heals/kb/recipe/bd4767eb-Oven%20Baked%20Ribs',
   'nutrients': {'calories': 220.9,
    'carbohydrates': 24.9,
    'fat': 11.7,
    'protein': 0.0},
   'ingredients': [{'name': 'thai sweet chili sauce',
     'uri': 'http://idea.rpi.edu/heals/kb/ingredientname/thai%20sweet%20chili%20sauce'},
    {'name': 'beef ribs',
     'uri': 'http://idea.rpi.edu/heals/kb/ingredientname/beef%20ribs'},
    {'name': 'cider vinegar',
     'uri': 'http://idea.rpi.edu/heals/kb/ingredientname/cider%20vinegar'},
    {'name': 'garlic cloves',
     'uri': 'http://idea.rpi.edu/heals/kb/ingredientname/garlic%20cloves'},
    {'name': 'green onions',
     'uri': 'http://idea.rpi.edu/heals/kb/ingredientname/green%20onions'},
    {'name': 'reduced sodium soy sauce',
     '

In [14]:
recipes = requests.post(f"{KBQA_URL}/recipes", json=kbqa_answer["answer_id_list"]).json()
for recipe in recipes:
    print(recipe['name'])
    for name, value in recipe['nutrients'].items():
        print(f"  {name}: {value}")
    print('  Ingredients:')
    for ing in recipe['ingredients']:
        print(f'  - {ing["name"]}')

Oven Baked Ribs
  calories: 220.9
  carbohydrates: 24.9
  fat: 11.7
  protein: 0.0
  Ingredients:
  - thai sweet chili sauce
  - beef ribs
  - cider vinegar
  - garlic cloves
  - green onions
  - reduced sodium soy sauce
  - smooth peanut butter
  - tomato sauce
  - treacle
  - vegetable oil
